# Design rationale

The [netCDF STF 2.0 compliant format](https://github.com/csiro-hydroinformatics/efts/blob/107c553045a37e6ef36b2eababf6a299e7883d50/docs/netcdf_for_water_forecasting.md) is such that a file loaded from Python via `xarray` is not the most convenient data model for users.

This notebook illustrates interactively the behaviors, and informs the design choices made to reconcile the `xarray` view with the on-disk representation.

## Loading an existing reference netCDF file

A file was created using (probably) a Matlab implementation of STF data handling and I/O. Let's load it via `xarray` as well as the `netCDF4` package, as we are not sure which will be most adequate for `efts-io` for saving/loading operations.

In [1]:
import xarray as xr

`xarray.open_dataset` has arguments to turn on/off the decoding of climate and forecast (SF) and related conventions. 

* `decode_times=False` is a must, otherwise the statement fails. Decoding would work for the `time` dimension, but decoding `lead_time` fails.  
* `decode_cf` seems to influence at least how the station_name variable appears, notably whether it ends up of dimensions `(station, strLen)` if True, or `(station,)` if False.

In [2]:
fn = "/home/per202/data/sf/sample/HT_swiftRain_daily_stfv2_2000111523+0000-2023111023+0000.nc"
rain_xr = xr.open_dataset(fn, decode_times=False, decode_cf=False)

In [3]:
import netCDF4 as nc

In [4]:
rain_nc = nc.Dataset(fn)

### xarray read

In [5]:
rain_xr

<xarray.Dataset> Size: 19MB
Dimensions:       (time: 8396, station: 563, lead_time: 1, strLen: 30,
                   ens_member: 1)
Coordinates:
  * time          (time) int32 34kB 1 2 3 4 5 6 ... 8392 8393 8394 8395 8396
  * station       (station) int32 2kB 1 2 3 4 5 6 7 ... 558 559 560 561 562 563
  * lead_time     (lead_time) int32 4B 0
  * ens_member    (ens_member) int32 4B 1
Dimensions without coordinates: strLen
Data variables:
    station_id    (station) int32 2kB ...
    station_name  (station, strLen) |S1 17kB ...
    lat           (station) float32 2kB ...
    lon           (station) float32 2kB ...
    area          (station) float32 2kB ...
    rain_obs      (time, ens_member, station, lead_time) float32 19MB ...
Attributes:
    title:                   Precip from Hydro Tasmania's observation network...
    institution:             CSIRO Land & Water
    source:                  
    catchment:               Hydro Tas
    STF_convention_version:  2.0
    STF_nc_spec:             https://wiki.csiro.au/display/wirada/NetCDF+for+...
    comment:                 
    history:                 2024-07-25 15:28:34 +10.0 - File created

If we use `decode_cf=True`, we seem to get a one dimensional array of array of bytes, rather than a matrix of bytes (type 'S1'):

In [6]:
rain_cfdecode = xr.open_dataset(fn, decode_times=False, decode_cf=True)

In [7]:
rain_cfdecode.station_name

<xarray.DataArray 'station_name' (station: 563)> Size: 17kB
[563 values with dtype=|S30]
Coordinates:
  * station  (station) int32 2kB 1 2 3 4 5 6 7 8 ... 557 558 559 560 561 562 563
Attributes:
    long_name:  station or node name

In [8]:
rain_cfdecode.station_name.values[1]

np.bytes_(b'19629011')

### netCDF4 read

In [9]:
rain_nc

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    title: Precip from Hydro Tasmania's observation network areally averaged with inverse distance squared weighting
    institution: CSIRO Land & Water
    source: 
    catchment: Hydro Tas
    STF_convention_version: 2.0
    STF_nc_spec: https://wiki.csiro.au/display/wirada/NetCDF+for+SWIFT
    comment: 
    history: 2024-07-25 15:28:34 +10.0 - File created
    dimensions(sizes): time(8396), station(563), lead_time(1), strLen(30), ens_member(1)
    variables(dimensions): int32 time(time), int32 station(station), int32 lead_time(lead_time), int32 station_id(station), |S1 station_name(station, strLen), int32 ens_member(ens_member), float32 lat(station), float32 lon(station), float32 area(station), float32 rain_obs(time, ens_member, station, lead_time)
    groups: 

Modulo the value of `decode_cf` for `xarray.open_dataset`, the shape of the data in memory appears consistent between `xarray` and `netCDF4`

## Requirements

### Desired in-memory representation

See [this discussion](https://github.com/csiro-hydroinformatics/efts-io/issues/2) for background.

We assume that an "intuitive" data representation in an xarray dataset would have the following characteristics:

* The `time` coordinate has values with python representations `np.datetime64` or similar
* A `station_id` coordinate has values as strings rather than bytes, so that slicing can be done with statements such as `data.sel(station_id="407113A")`. The STF representation is such that `station` is a dimension/coordinate, not `station_id`
* In the example case loaded, the variable datatypes is 32-bits `np.float32` rather than 64 bits `np.float`. The latter is probably more convenient in most use cases we can anticipate. However we may want to consider keeping a 32 bits representation: ensemble forecasting and modelling methods can be RAM-hungry even with 2024 typical machine setups.
* coordinate data is in type `int32`. Memory footprint is not a consideration, we may want to change is to 64 bits, or not, based on other factors.
* There should be a coordinate named "realisation" (or U.S. "realization"??) rather than "ens_member" 

### STF 2.0 compliance

It is imperative to be able to export the in-memory xarray representation to a `netCDF` file that complies with documented conventions and is readable by existing toolsets in `Matlab`, `C++` or even `Fortran`. A key question is whether we can use `xarray.to_netcdf` or whether we need to use the lower level package `netCDF4` to achieve that.


## Implementation

This notebook will illustrate the various steps taken to bridge the gap between the on-disk and in-memory representations.

### Reading from disk


In [29]:
rain_cfdecode

<xarray.Dataset> Size: 19MB
Dimensions:       (time: 8396, station: 563, lead_time: 1, ens_member: 1)
Coordinates:
  * time          (time) int32 34kB 1 2 3 4 5 6 ... 8392 8393 8394 8395 8396
  * station       (station) int32 2kB 1 2 3 4 5 6 7 ... 558 559 560 561 562 563
  * lead_time     (lead_time) int32 4B 0
  * ens_member    (ens_member) int32 4B 1
Data variables:
    station_id    (station) int32 2kB ...
    station_name  (station) |S30 17kB b'18594010' b'19629011' ... b'28294677'
    lat           (station) float32 2kB ...
    lon           (station) float32 2kB ...
    area          (station) float32 2kB ...
    rain_obs      (time, ens_member, station, lead_time) float32 19MB ...
Attributes:
    title:                   Precip from Hydro Tasmania's observation network...
    institution:             CSIRO Land & Water
    source:                  
    catchment:               Hydro Tas
    STF_convention_version:  2.0
    STF_nc_spec:             https://wiki.csiro.au/display/wirada/NetCDF+for+...
    comment:                 
    history:                 2024-07-25 15:28:34 +10.0 - File created

#### time

For background in issue https://jira.csiro.au/browse/WIRADA-635. We cannot have xarray automagically decoding this axis, so we need to do the work manually, but using as much as possible work already done. Not sure how I had figured out about `CFDatetimeCoder`, but:

In [30]:
from xarray.coding import times

In [12]:
decod = times.CFDatetimeCoder(use_cftime=True)

In [13]:
decod.decode?

Signature: decod.decode(variable: 'Variable', name: 'T_Name' = None) -> 'Variable'
Docstring: Convert an decoded variable to a encoded variable
File:      ~/src/efts-io/.venv/lib/python3.12/site-packages/xarray/coding/times.py
Type:      method

We need to pass a "Variable", not a `DataArray`

In [14]:
type(rain_cfdecode.coords['time'])

xarray.core.dataarray.DataArray

In [15]:
TIME_DIMNAME="time"
var = xr.as_variable(rain_cfdecode.coords[TIME_DIMNAME])

In [16]:
time_zone = var.attrs["time_standard"]
time_coords = decod.decode(var, name=TIME_DIMNAME)

In [17]:
time_zone

'UTC'

In [18]:
timestamp = time_coords.values[0]
timestamp

cftime.DatetimeGregorian(2000, 11, 15, 23, 0, 0, 0, has_year_zero=False)

Date/time, calendar and time zone handling are a topic of underappreciated complexity, to put it mildly. Let's look at what we get here.

Unfamiliar with this type of time stamp. It seems not to have time zone from the decoding operation, but can have it:

In [19]:
timestamp.tzinfo is None

True

Should our new `time` axis hold time zone info with each time stamp, or still rely on the coordinate attribute `time_standard`? 

In [20]:
from efts_io.wrapper import cftimes_to_pdtstamps

In [21]:
new_time_values = cftimes_to_pdtstamps(
    time_coords.values,
    time_zone,
)
new_time_values

array([Timestamp('2000-11-15 23:00:00+0000', tz='UTC'),
       Timestamp('2000-11-16 23:00:00+0000', tz='UTC'),
       Timestamp('2000-11-17 23:00:00+0000', tz='UTC'), ...,
       Timestamp('2023-11-08 23:00:00+0000', tz='UTC'),
       Timestamp('2023-11-09 23:00:00+0000', tz='UTC'),
       Timestamp('2023-11-10 23:00:00+0000', tz='UTC')], dtype=object)

This may be a suitable time axis. Depending on usage needs we may want to revisit though. In particular, users may create "naive" date time stamps from strings: how would `ds.sel()` then behave if time stamps have time zones??

In [24]:
import pandas as pd
pd.Timestamp('2000-11-15 23:00:00+0000')

Timestamp('2000-11-15 23:00:00+0000', tz='UTC')

In [25]:
pd.Timestamp('2000-11-15 23:00:00+0000') == new_time_values[0]

True

In [27]:
pd.Timestamp('2000-11-15 23:00:00')

Timestamp('2000-11-15 23:00:00')

In [28]:
pd.Timestamp('2000-11-15 23:00:00') == new_time_values[0]

False

As expected, the naive date time is not equal to the one with a time zone. Using time zone in the time stamps may be a fraught choice in practice. In particular there may be logical but unintuitive if we use a time `slice` to subset data

See also [github issue 3](https://github.com/csiro-hydroinformatics/efts-io/issues/3)

In [31]:
new_time_values = cftimes_to_pdtstamps(
    time_coords.values,
    None,
)
new_time_values

array([Timestamp('2000-11-15 23:00:00'), Timestamp('2000-11-16 23:00:00'),
       Timestamp('2000-11-17 23:00:00'), ...,
       Timestamp('2023-11-08 23:00:00'), Timestamp('2023-11-09 23:00:00'),
       Timestamp('2023-11-10 23:00:00')], dtype=object)

#### station_id